In [1]:
import os
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
import tensorflow_datasets as tfds
import numpy as np


Use Masking Layer to create an ImageNet that only predicts for Stanford Dogs dataset.
Need mapping list of indices between full ImageNet and Stanford Dogs.

Concept:
start with existing Stanford Dogs notebook about transfer learning.
Need mapping list of indices between full ImageNet and Stanford Dogs.

Load ImageNet model.
Download any subset of ImageNet with original indexes, preferably a mix with some dogs.
Predict from subset
Display misclassified image pairs. 


Stanford Dogs section
Create 2-way mapping tables
Predict SD only:
Create Dataset which maps index to ImageNet index
Predict Stanford Dogs from ImageNet with evaluate.
Predict again, pulling logits, counting misclassified outside of SD index list.
Display misclassified outside 

Save weights to file.
Pop final Dense layer.
Get name and config params, verify activation = softmax
Create new Dense layer with same shape and name, no activation.
Create simple Model with just Dense layer
'load weights by name' into new Model
now have Dense with no activation
Add Dense to ImageNet model.
Add Masking to ImageNet model.
Add Softmax layer.



In [2]:

IMG_SIZE = 224
IMAGENET_CLASSES = 1000
batch_size = 64

dataset_name = "stanford_dogs"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
SD_CLASSES = ds_info.features["label"].num_classes
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))
# forget input preprocessing for simplicity
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

...

  0%|          | 0/12000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/stanford_dogs/0.2.0.incompleteMMS5OQ/stanford_dogs-test.tfrecord


  0%|          | 0/8580 [00:00<?, ? examples/s]

Dataset stanford_dogs downloaded and prepared to /root/tensorflow_datasets/stanford_dogs/0.2.0. Subsequent calls will reuse this data.


The Stanford Dogs image set is a subset of ImageNet-1000, and occupies a few ranges of indexes in the larger dataset. (Stanford Dogs does not include several wild canid species, sticking entirely to domestic breeds.)

In [3]:
def sd2imagenet(index):
    if index >= 0 and index <= 99:
        return index + 151
    elif index >= 100 and index <= 116:
        return index + 152
    elif index >= 117 and index <= 119:
        return index + 156
    else:
        return -1

def imagenet2sd(index):
    if index >= 151 and index <= 250:
        return index - 151
    elif index >= 252 and index <= 268:
        return index - 152
    elif index >= 273 and index <= 275:
        return index - 156
    else:
        return -1


In [4]:
def build_masked_model(mapping):
    model = EfficientNetB0(weights='imagenet')
    # for l in model.layers:
    #     print(l.name)
    # model.predict(np.zeros((1, IMG_SIZE, IMG_SIZE, 3)))
    # print(model)
    # print(model.inputs)
    # print(model.outputs)

    # Freeze the pretrained weights
    # model.trainable = False

    # model_file = '/tmp/model_weights.h5'
    # model.save_weights(model_file)

    probs = model.layers.pop()
    print(type(probs))
    probs_input_size = probs.input_shape[1]
    probs_output_size = probs.output_shape[1]
    probs_weights = probs.get_weights()
    probs_config = probs.get_config()
    print(probs_config)
    # probs_config['trainable'] = True
    probs_config['activation'] = 'linear'

    probs = tf.keras.layers.Dense(**probs_config)
    # print(probs.get_config())

    # probs_model = tf.keras.Model()
    probs_inputs = tf.keras.layers.Input(shape=(probs_input_size))
    probs_outputs = tf.keras.layers.Dense((probs_output_size))

    probs_model = tf.keras.Sequential()
    probs_model.add(probs_inputs)
    probs_model.add(probs)
    probs_model.add(probs_outputs)
    probs_model.compile(optimizer='adam')
    # probs_model = tf.keras.Model(probs_inputs, probs_outputs, name='mini')
    probs_model.predict(np.zeros((1, probs_input_size), dtype='float32'))
    probs.set_weights(probs_weights)
    # os.remove(model_file)

    masks = [0.0 for x in range(probs_output_size)]
    for index in mapping:
        masks[index] = 1.0
    mask_layer = tf.keras.layers.Lambda(lambda x: x * masks)
    mask_layer.trainable = False
    probs_model.add(mask_layer)
    probs_model.compile(optimizer='adam')

    preds = probs_model.predict(np.ones((1, probs_input_size), dtype='float32'))
    print(preds[0][:8])
   
    return probs_model

m = build_masked_model([3, 6])

21848064/21834768 [==============================] - 0s 0us/step
<class 'keras.layers.core.dense.Dense'>
{'name': 'predictions', 'trainable': True, 'dtype': 'float32', 'units': 1000, 'activation': 'softmax', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 0.3333333333333333, 'mode': 'fan_out', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}
[ 0.         -0.          0.         -0.7784309  -0.         -0.
 -0.57869166  0.        ]


In [5]:
def build_masked_model(mapping):
    model = EfficientNetB0(weights='imagenet')
    model.predict(np.zeros((1, IMG_SIZE, IMG_SIZE, 3)))
    # print(model)
    # print(model.inputs)
    # print(model.outputs)

    # Freeze the pretrained weights
    model.trainable = False

    # model_file = '/tmp/model_weights.h5'
    # model.save_weights(model_file)

    probs = model.layers.pop()
    probs_input_size = probs.input_shape[1]
    probs_output_size = probs.output_shape[1]
    probs_config = probs.get_config()
    probs_config['activation'] = 'linear'

    probs = tf.keras.layers.Dense(**probs_config)
    print(probs.get_config())

    # probs_model = tf.keras.Model()
    probs_inputs = tf.keras.layers.Input(shape=(probs_input_size))
    probs_outputs = tf.keras.layers.Dense((probs_output_size))

    probs_model = tf.keras.Sequential()
    probs_model.add(probs_inputs)
    probs_model.add(probs)
    probs_model.add(probs_outputs)
    probs_model.compile(optimizer='adam')
    probs_model.predict(np.zeros((1, probs_input_size), dtype='float32'))
    # probs_model.load_weights(model_file, by_name=True)
    # os.remove(model_file)

    masks = [0.0 for x in range(probs_output_size)]
    for index in mapping:
        masks[index] = 1.0
    mask_layer = tf.keras.layers.Lambda(lambda x: x * masks)
    mask_layer.trainable = False
    probs_model.add(mask_layer)
    probs_model.compile(optimizer='adam')

    preds = probs_model.predict(np.ones((1, probs_input_size), dtype='float32'))
    # print(preds)
   
    return probs_model

m = build_masked_model([])

{'name': 'predictions', 'trainable': False, 'dtype': 'float32', 'units': 1000, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 0.3333333333333333, 'mode': 'fan_out', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}


NameError: ignored

In [ ]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()


# plot_hist(hist)

In [ ]:
dataset, info = tfds.load(name="stanford_dogs", with_info=True)

get_name = info.features['label'].int2str

for doggo in dataset['train'].take(10):
    plt.figure()
    plt.imshow(doggo['image'])
    plt.title(get_name(doggo['label']))

In [ ]:
labels = {}
max_label = -1
min_label = 100000000
for doggo in dataset['train']: #.take(100):
    # print(type(doggo))
    index = int(doggo['label'].numpy())
    # print(type(index))
    # print(dir(index))
    # print(index)
    if index > max_label:
        max_label = index
    if index < min_label:
        min_label = index
    labels[index] = get_name(doggo['label'])
print(min_label)
print(max_label)
print(len(labels))
for x in range(len(labels)):
    print(x, labels[x])